## 로이터 뉴스 토픽 분류: 다중 클래스 분류

이 노트북은 Keras를 사용하여 로이터 뉴스 데이터셋을 46개의 상호 배타적인 토픽으로 분류하는 신경망을 구축합니다. 이는 다중 클래스 분류(multi-class classification) 문제의 전형적인 예시입니다.

### 1. 라이브러리 임포트 및 데이터 로드
필요한 라이브러리를 임포트하고 Keras에 내장된 로이터 데이터셋을 로드합니다. 데이터셋은 빈도가 높은 10,000개의 단어만 사용하도록 제한합니다 (`num_words=10000`).

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import reuters
from keras import models, layers
from keras.utils import to_categorical
import matplotlib.pyplot as plt

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print("--- 데이터 형태 ---")
print("훈련 데이터 수:", len(train_data))
print("테스트 데이터 수:", len(test_data))
print("훈련 데이터 형태:", train_data.shape)
print("훈련 레이블 형태:", train_labels.shape)

### 2. 데이터 탐색
데이터가 어떻게 구성되어 있는지 확인합니다. 각 샘플은 단어 인덱스의 리스트로 구성되어 있습니다. 첫 번째 훈련용 뉴스가 어떤 단어들로 이루어져 있는지 디코딩하여 확인해봅니다.

In [ ]:
print("첫 번째 훈련 데이터 (정수 인덱스):")
print(train_data[0])

# 단어 인덱스 로드
word_index = reuters.get_word_index()
# 정수 -> 단어 매핑
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# 리뷰 디코딩. 0, 1, 2는 '패딩', '문서 시작', '사전에 없음'을 위한 인덱스이므로 3을 뺌
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

print("\n첫 번째 훈련 데이터 (디코딩된 텍스트):")
print(decoded_review)

### 3. 데이터 전처리
신경망에 데이터를 입력하기 위해 정수 시퀀스를 고정된 크기의 텐서로 변환해야 합니다.

1.  **입력 데이터**: 각 뉴스를 10,000차원의 벡터로 변환합니다 (원-핫 인코딩과 유사). 특정 단어 인덱스가 뉴스에 존재하면 해당 위치는 1이 되고, 나머지는 0이 됩니다.
2.  **레이블 데이터**: 정수 형태의 레이블(0~45)을 46차원의 원-핫 벡터로 변환합니다. (`categorical_crossentropy` 손실 함수를 사용하기 위함)

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # 해당 인덱스 위치를 1로 만듦
    return results

# 훈련 및 테스트 데이터 벡터 변환
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# 레이블 원-핫 인코딩
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

print("\n벡터화된 훈련 데이터 형태:", x_train.shape)
print("원-핫 인코딩된 훈련 레이블 형태:", one_hot_train_labels.shape)
print("\n첫 번째 원-핫 인코딩된 레이블:")
print(one_hot_train_labels[0])

### 4. 신경망 모델 구축
분류 문제를 위한 `Sequential` 모델을 정의합니다.

- **입력층/은닉층**: `relu` 활성화 함수를 사용하는 `Dense` 층 2개를 사용합니다. 64개의 유닛을 가진 은닉층은 너무 작지도, 크지도 않은 적절한 크기입니다.
- **출력층**: 46개의 클래스를 분류해야 하므로 46개의 유닛을 가지며, 각 클래스에 대한 확률 분포를 출력하기 위해 `softmax` 활성화 함수를 사용합니다.

In [ ]:
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(10000,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(46, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

### 5. 훈련 검증 데이터 분리
훈련 중 모델의 성능을 모니터링하고 과대적합을 확인하기 위해, 기존 훈련 데이터에서 1,000개의 샘플을 검증 세트로 분리합니다.

In [ ]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

### 6. 모델 훈련
`fit()` 메서드를 사용하여 모델을 학습시킵니다. `validation_data` 인자로 검증 세트를 전달하여 각 에포크마다 성능을 측정합니다.

In [ ]:
# 에포크 수를 줄여서 실행 (원본: 500)
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20, # 과적합이 빨리 일어나므로 20으로 조정
                    batch_size=512,
                    validation_data=(x_val, y_val))

### 7. 훈련 과정 시각화
훈련 및 검증 과정에서의 손실(loss)과 정확도(accuracy)를 그래프로 그려 과대적합이 발생하는 시점을 확인합니다.

In [ ]:
history_dict = history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

epochs = range(1, len(loss) + 1)

# 손실 그래프
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

그래프를 보면, 훈련 손실은 계속 감소하고 훈련 정확도는 계속 증가하는 반면, 검증 손실과 검증 정확도는 특정 에포크(대략 9~10) 이후로 성능이 저하되거나 정체됩니다. 이는 모델이 훈련 데이터에 과대적합되고 있음을 의미합니다.

### 8. 모델 평가 및 예측
테스트 데이터로 최종 모델의 성능을 평가하고, 일부 샘플에 대한 예측 결과를 실제 정답과 비교해봅니다.

In [ ]:
print("--- 최종 모델 평가 ---")
results = model.evaluate(x_test, one_hot_test_labels)
print(f"테스트 손실: {results[0]:.4f}")
print(f"테스트 정확도: {results[1]:.4f}")

# 테스트 데이터에 대한 예측
predictions = model.predict(x_test)

print("\n--- 예측 결과 샘플 ---")
for i in range(10):
    predicted_class = np.argmax(predictions[i])
    actual_class = test_labels[i]
    print(f"[샘플 {i}] 예측: {predicted_class}, 실제: {actual_class}")